## TF

In [118]:
collection = [
    ('Document1', 'This is a sample'),
    ('Document2', 'This is another sample')
]

# in-memory (Hash Key 값)
# 전체 색인어 목록(Dictionary)
# {단어1:포스팅위치, 단어2:포스팅위치, ...}
globalLexicon = dict()

# 전체 문서 목록(Dictionary)
# [0:문서1, 1:문서2, ...]
globalDocument = list()

# disk
# 사전에 있는 색인어 중, 어느 문서에서, 몇 번 나타났는지
# [(단어 idx, 문서 idx, 빈도, 다음주소), ...]
# [0:Tuple(lexiconIdx, documentIdx, freq, 다음포스팅위치-fptr)]
# 메모리 X, File OK
globalPosting = list()

In [119]:
for (docName, docContent) in collection:
    # Pointer 대체용, Key, Document 이름은 절대로 겹치지 않는다는 가정
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    # {단어idx:빈도, 단어idx:빈도, ...}
    localPosting = dict()
    
    # Local 작업
    for term in docContent.lower().split():
        # Local에 대해서, 없으면 추가
        if term not in localPosting.keys():
            localPosting[term] = 1
        # 있으면, 빈도 증가
        else:
            localPosting[term] += 1
     
    # Global Marge
    # fp -> struct(단어, 빈도) (localPosting)
    for indexTerm, termFreq in localPosting.items():
        if indexTerm not in globalLexicon.keys(): 
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting) # fseek
            postingData = (lexiconIdx, docIdx, termFreq, -1)
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx # globalPosting 위치(ptr:idx)
        else: # 기존 단어의 idx 가져오기
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = (lexiconIdx, docIdx, termFreq, beforeIdx)
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx
            
#     print(localPosting)
# print(globalDocument)

#         if term not in globalLexicon.keys():
#             lexiconIdx = len(globalLexicon) 0

In [120]:
globalLexicon, globalDocument

({'this': 4, 'is': 5, 'a': 2, 'sample': 7, 'another': 6},
 ['Document1', 'Document2'])

In [121]:
globalPosting

[(0, 0, 1, -1),
 (1, 0, 1, -1),
 (2, 0, 1, -1),
 (3, 0, 1, -1),
 (0, 1, 1, 0),
 (1, 1, 1, 1),
 (4, 1, 1, -1),
 (3, 1, 1, 3)]

In [122]:
for indexTerm, postingIdx in globalLexicon.items():
    # indexTerm:단어: postingIdx:위치, ...
    print(indexTerm)
    
    while True: # Posting Next:-1
        if postingIdx == -1:
            break
            
        postingData = globalPosting[postingIdx]
        print('  DocName:{0} - TermFreq:{1} - Next:{2}'.format(globalDocument[postingData[1]], postingData[2], postingData[3]))
        postingIdx = postingData[3]
        
    print()

this
  DocName:Document2 - TermFreq:1 - Next:0
  DocName:Document1 - TermFreq:1 - Next:-1

is
  DocName:Document2 - TermFreq:1 - Next:1
  DocName:Document1 - TermFreq:1 - Next:-1

a
  DocName:Document1 - TermFreq:1 - Next:-1

sample
  DocName:Document2 - TermFreq:1 - Next:3
  DocName:Document1 - TermFreq:1 - Next:-1

another
  DocName:Document2 - TermFreq:1 - Next:-1



In [123]:
globalPosting[globalLexicon['sample']]

(3, 1, 1, 3)

In [124]:
globalPosting[globalPosting[globalLexicon['sample']][3]]   # 다음 주소가 "-1" 일때 까지 반복해서 찾음

(3, 0, 1, -1)

## TF-IDF

In [125]:
collection = [
    ('Document1', 'This is a a a a a a a a a a sample'),
    ('Document2', 'This is a sample'),
]

### TF

In [126]:
from math import log10

def binaryTF(freq):
    if freq > 0:
        return 1
    else:
        return 0

def rawTF(freq):
    return freq

def basicTF(freq, totalFreq):
    return freq/totalFreq

def logTF(freq):
    if freq > 0:
        return 1+log10(freq)
    else:
        return 0

def doubleNormalTF(K, freq, maxFreq): 
    return K + ((1-K) * (freq/maxFreq))

In [127]:
for (docName, docContent) in collection:
    localPosting = dict()
    
    for term in docContent.lower().split():
        if term not in localPosting.keys():
            localPosting[term] = 1
        else:
            localPosting[term] += 1
    
    # localPosting => {단어:빈도, 단어:빈도, ...}
    
    maxFreq = max(localPosting.values())
    totalCount = sum(localPosting.values())
    
    print('-----------------------------------')
    
    for term, freq in localPosting.items():
        print(term)
        print('1. Binary:{0}'.format(binaryTF(freq)))
        print('2. Raw:{0}'.format(rawTF(freq)))
        print('3. Basic:{0}'.format(basicTF(freq, totalCount)))
        print('4. Log:{0}'.format(logTF(freq)))
        print('5. DoubleNormalization:{0}'.format(doubleNormalTF(0, freq, maxFreq)))
        print('6. DoubleNormalization:{0}'.format(doubleNormalTF(0.5, freq, maxFreq)))
        print()
    
    print(localPosting)
    print()

-----------------------------------
this
1. Binary:1
2. Raw:1
3. Basic:0.07692307692307693
4. Log:1.0
5. DoubleNormalization:0.1
6. DoubleNormalization:0.55

is
1. Binary:1
2. Raw:1
3. Basic:0.07692307692307693
4. Log:1.0
5. DoubleNormalization:0.1
6. DoubleNormalization:0.55

a
1. Binary:1
2. Raw:10
3. Basic:0.7692307692307693
4. Log:2.0
5. DoubleNormalization:1.0
6. DoubleNormalization:1.0

sample
1. Binary:1
2. Raw:1
3. Basic:0.07692307692307693
4. Log:1.0
5. DoubleNormalization:0.1
6. DoubleNormalization:0.55

{'this': 1, 'is': 1, 'a': 10, 'sample': 1}

-----------------------------------
this
1. Binary:1
2. Raw:1
3. Basic:0.25
4. Log:1.0
5. DoubleNormalization:1.0
6. DoubleNormalization:1.0

is
1. Binary:1
2. Raw:1
3. Basic:0.25
4. Log:1.0
5. DoubleNormalization:1.0
6. DoubleNormalization:1.0

a
1. Binary:1
2. Raw:1
3. Basic:0.25
4. Log:1.0
5. DoubleNormalization:1.0
6. DoubleNormalization:1.0

sample
1. Binary:1
2. Raw:1
3. Basic:0.25
4. Log:1.0
5. DoubleNormalization:1.0
6. Doub

## IDF

In [183]:
collection = [
    ('Document1', 'This a a a a a a a a a a  sample'),
    ('Document2', 'This is a sample'),
]

In [184]:
def unaryIDF():
    return 1

def basicIDF(N, df):
    return log10(N/df)

def smoothigIDF(N, df):
    return log10((N+1)/df)

def probabilityIDF(N, df):
    return log10((N-df+1)/df)

In [185]:
N = len(collection)

for term, ptr in globalLexicon.items():
    # term:단어, ptr:위치, ...
    df = 0
    
    while True:    # ptr Next: -1
        if ptr == -1:
            break
        
        df += 1
        postingData = globalPosting[ptr]
        ptr = postingData[3]
    
    print(term)
    print('1. UnaryIDF: {0}'.format(unaryIDF()))
    print('2. BasicIDF: {0}'.format(basicIDF(N,df)))
    print('3. SmoothigIDF: {0}'.format(smoothigIDF(N,df)))
    print('4. ProbabilityIDF: {0}'.format(probabilityIDF(N,df)))
    print()

this
1. UnaryIDF: 1
2. BasicIDF: 0.3010299956639812
3. SmoothigIDF: 0.47712125471966244
4. ProbabilityIDF: 0.3010299956639812

is
1. UnaryIDF: 1
2. BasicIDF: 0.3010299956639812
3. SmoothigIDF: 0.47712125471966244
4. ProbabilityIDF: 0.3010299956639812

a
1. UnaryIDF: 1
2. BasicIDF: 0.3010299956639812
3. SmoothigIDF: 0.47712125471966244
4. ProbabilityIDF: 0.3010299956639812

sample
1. UnaryIDF: 1
2. BasicIDF: 0.3010299956639812
3. SmoothigIDF: 0.47712125471966244
4. ProbabilityIDF: 0.3010299956639812

another
1. UnaryIDF: 1
2. BasicIDF: 0.3010299956639812
3. SmoothigIDF: 0.47712125471966244
4. ProbabilityIDF: 0.3010299956639812

not
1. UnaryIDF: 1
2. BasicIDF: 0.3010299956639812
3. SmoothigIDF: 0.47712125471966244
4. ProbabilityIDF: 0.3010299956639812



In [186]:

collection = [
    ('Document1', 'This is a sample'),
    ('Document2', 'This is another sample'),
    ('Document3', 'This is not sample'),
    ('Document4', 'a not sample'),
    ('Document5', 'not'),
    ('Document5', 'not sample')
]

globalLexicon = dict()
globalDocument = list()
globalPosting = list()

for (docName, docContent) in collection:
    # Pointer 대체용, Key, Document 이름은 절대로 겹치지 않는다는 가정
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    # {단어idx:빈도, 단어idx:빈도, ...}
    localPosting = dict()

    for token in docContent.lower().split():
        if token not in localPosting.keys():
            localPosting[token] = 1
        else:
            localPosting[token] += 1
            
    maxFreq = max(localPosting.values())
     
    # Global Marge
    # fp -> struct(단어, 빈도) (localPosting)
    for token, Freq in localPosting.items():
        if token not in globalLexicon.keys(): 
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting) # fseek
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, freq, maxFreq), -1]
            globalPosting.append(postingData)
            globalLexicon[token] = postingIdx # globalPosting 위치(ptr:idx)
        else: # 기존 단어의 idx 가져오기
            # tuple로 리턴받을경우 immutable(읽기전용)이라 list로 바꿔 주어야 함. 
            lexiconIdx = list(globalLexicon.keys()).index(token)
            postingIdx = len(globalPosting)
            beforeIdx = globalLexicon[token]
            postingData = [lexiconIdx, docIdx, doubleNormalTF(0, freq, maxFreq), -1]
            globalPosting.append(postingData)
            globalLexicon[token] = postingIdx
            
#     print(localPosting)
# print(globalDocument)

#         if term not in globalLexicon.keys():
#             lexiconIdx = len(globalLexicon) 0

In [187]:
globalPosting

[[0, 0, 1.0, -1],
 [1, 0, 1.0, -1],
 [2, 0, 1.0, -1],
 [3, 0, 1.0, -1],
 [0, 1, 1.0, -1],
 [1, 1, 1.0, -1],
 [4, 1, 1.0, -1],
 [3, 1, 1.0, -1],
 [0, 2, 1.0, -1],
 [1, 2, 1.0, -1],
 [5, 2, 1.0, -1],
 [3, 2, 1.0, -1],
 [2, 3, 1.0, -1],
 [5, 3, 1.0, -1],
 [3, 3, 1.0, -1],
 [5, 4, 1.0, -1],
 [5, 5, 1.0, -1],
 [3, 5, 1.0, -1]]

In [188]:


N = len(globalDocument)
idfList = dict()

for term, ptr in globalLexicon.items():
    # term:단어, ptr:위치, ...
    df = 0
    oldptr = ptr
    
    while True:    # ptr Next: -1
        if ptr == -1:
            break
        
        df += 1
        postingData = globalPosting[ptr]
        ptr = postingData[3]
        
    ptr = oldptr
    idf = basicIDF(N, df)
    idfList[term] = idf # 나중에 단어가 나왔을때 idf를 불러올 수 있도록 하기 위함
    print("단어 : {0} / IDF : {1}".format(term, idf))
    
    while True:
        if ptr == -1:
            break
            
        postingData = globalPosting[ptr]
        tf = postingData[2]
        postingData[2] = tf * idf
        ptr = postingData[3]
        print(" 문서 : {0} / TF : {1} / TF-IDF : {2}".format(
            globalDocument[postingData[1]],
            tf, postingData[2]))
    idf = basicIDF(N, df)

단어 : this / IDF : 0.7781512503836436
 문서 : Document3 / TF : 1.0 / TF-IDF : 0.7781512503836436
단어 : is / IDF : 0.7781512503836436
 문서 : Document3 / TF : 1.0 / TF-IDF : 0.7781512503836436
단어 : a / IDF : 0.7781512503836436
 문서 : Document4 / TF : 1.0 / TF-IDF : 0.7781512503836436
단어 : sample / IDF : 0.7781512503836436
 문서 : Document5 / TF : 1.0 / TF-IDF : 0.7781512503836436
단어 : another / IDF : 0.7781512503836436
 문서 : Document2 / TF : 1.0 / TF-IDF : 0.7781512503836436
단어 : not / IDF : 0.7781512503836436
 문서 : Document5 / TF : 1.0 / TF-IDF : 0.7781512503836436


In [189]:
globalPosting

[[0, 0, 1.0, -1],
 [1, 0, 1.0, -1],
 [2, 0, 1.0, -1],
 [3, 0, 1.0, -1],
 [0, 1, 1.0, -1],
 [1, 1, 1.0, -1],
 [4, 1, 0.7781512503836436, -1],
 [3, 1, 1.0, -1],
 [0, 2, 0.7781512503836436, -1],
 [1, 2, 0.7781512503836436, -1],
 [5, 2, 1.0, -1],
 [3, 2, 1.0, -1],
 [2, 3, 0.7781512503836436, -1],
 [5, 3, 1.0, -1],
 [3, 3, 1.0, -1],
 [5, 4, 1.0, -1],
 [5, 5, 0.7781512503836436, -1],
 [3, 5, 0.7781512503836436, -1]]

In [190]:
def euclidian(x, y):
    return (x-y) ** 2

In [218]:
query = "a"
queryPosting = dict()
    
for token in query.lower().split():
    if token not in queryPosting.keys():
        queryPosting[token] = 1
    else:
        queryPosting[token] += 1

# localPosting => {단어:빈도, 단어:빈도, ...}

maxFreq = max(queryPosting.values())

for token, freq in queryPosting.items():
    queryPosting[token] = doubleNormalTF(0.5, freq, maxFreq) * idfList[token]

In [219]:
queryPosting

{'a': 0.7781512503836436}

In [220]:
candidateList = dict()

for term, ptr in globalLexicon.items():
    queryWeight = 0
    
    if term in queryPosting.keys(): # keys에 있는지 검사 후 query가 있을 경우에 한해 처리하기 위함
        queryWeight = queryPosting[term]
        
    while True:    # ptr Next: -1
        if ptr == -1:
            break
        postingData = globalPosting[ptr]
        ptr = postingData[3]
        
        # 키 에러 방지용
        if postingData[1] in candidateList.keys():
            candidateList[postingData[1]] += euclidian(queryWeight, postingData[2])
        else:
            candidateList[postingData[1]] = euclidian(queryWeight, postingData[2])

In [230]:
# 정렬 (reversed = False는 작은값부터 큰 값 순서)
resultList = sorted(candidateList.items(), key=lambda x:x[1])

print(collection) # 검색하고자 하는 문서 이름과 내용
print("Current query :",  [query])

for i, (fileIdx, distance) in enumerate(resultList):
    print(query)
    print("순위 : {0} / 문서 : {1} / 거리 : {2}".format(
        (i+1), globalDocument[fileIdx], distance
    ))
    print(collection[fileIdx][1]) # 해당하는 문서도 출력
    
# query가 not일때와 a일때를 비교해 보면 유클리디안 거리의 문제점이 드러난다.(원노트 참고) 따라서 cosine simmilarity를 주로 사용. 

[('Document1', 'This is a sample'), ('Document2', 'This is another sample'), ('Document3', 'This is not sample'), ('Document4', 'a not sample'), ('Document5', 'not'), ('Document5', 'not sample')]
Current query : ['a']
a
순위 : 1 / 문서 : Document4 / 거리 : 0.0
a not sample
a
순위 : 2 / 문서 : Document2 / 거리 : 0.6055193684736281
This is another sample
a
순위 : 3 / 문서 : Document3 / 거리 : 1.2110387369472562
This is not sample
a
순위 : 4 / 문서 : Document5 / 거리 : 1.2110387369472562
not sample
